# Creating the final merged datasets

In [ ]:
import pandas as pd

## Loading and cleaning the bitcoin dataset for the merge

In [ ]:
hourly_bitcoin = pd.read_csv('/content/drive/MyDrive/FYP/dataset/Bitcoin/bitcoin_data.csv')

In [ ]:
hourly_bitcoin

In [ ]:
hourly_bitcoin.drop(columns=['close_time', 'quote_asset_volume', 'num_trades', 'taker_buy_base_asset_volume', 'taker_buy_quote_asset_volume', 'ignore'], inplace=True)


In [ ]:
# Remove duplicate rows
hourly_bitcoin = hourly_bitcoin.drop_duplicates()

# Remove rows with NaN values
hourly_bitcoin = hourly_bitcoin.dropna()

In [ ]:
hourly_bitcoin

In [ ]:
hourly_bitcoin = hourly_bitcoin.set_index('timestamp')

In [ ]:
hourly_bitcoin

## Loading, cleaning the sentiment analyzed twitter dataset and meging with the historical bitcoin data 

In [ ]:
cleaned_tweets = pd.read_csv('/content/drive/MyDrive/FYP/dataset/Twitter/twitter_data_sentiment_complete_2021.csv')

In [ ]:
cleaned_tweets

In [ ]:
cleaned_tweets.drop('text_clean', axis=1)


In [ ]:
type(cleaned_tweets)

In [ ]:
cleaned_tweets = cleaned_tweets.set_index('datetime')

In [ ]:
cleaned_tweets

In [ ]:
cleaned_tweets.index = pd.to_datetime(cleaned_tweets.index)  # convert index to datetime if it's not already
hourly_tweets = cleaned_tweets.resample('H').mean()

In [ ]:
hourly_tweets

In [ ]:
# Remove duplicate rows
hourly_tweets = hourly_tweets.drop_duplicates()

# Remove rows with NaN values
hourly_tweets = hourly_tweets.dropna()

In [ ]:
hourly_tweets

In [ ]:
hourly_tweets.index = hourly_tweets.index.astype(str).str.slice(stop=-6)
hourly_tweets

In [ ]:
final_hourly_twitter_bitcoin_merged_df = pd.merge(hourly_tweets, hourly_bitcoin, left_index=True, right_index=True, how='outer')

In [ ]:
final_hourly_twitter_bitcoin_merged_df

In [ ]:
# Remove duplicate rows
final_hourly_twitter_bitcoin_merged_df = final_hourly_twitter_bitcoin_merged_df.drop_duplicates()

# Remove rows with NaN values
final_hourly_twitter_bitcoin_merged_df = final_hourly_twitter_bitcoin_merged_df.dropna()

In [ ]:
final_hourly_twitter_bitcoin_merged_df

In [ ]:
final_hourly_twitter_bitcoin_merged_df.to_csv('/content/drive/MyDrive/FYP/dataset/Merged Dataset/twitter_bitcoin_merged.csv')

### testing tweet volume

In [ ]:
cleaned_tweets = pd.read_csv('/content/drive/MyDrive/FYP/dataset/Twitter/twitter_data_sentiment_complete_2021.csv')

In [ ]:
cleaned_tweets

In [ ]:
cleaned_tweets['datetime'] = pd.to_datetime(cleaned_tweets['datetime'])
tweet_vol = cleaned_tweets.groupby(pd.Grouper(key='datetime', freq='H')).size().reset_index(name='tweet_count')

In [ ]:
tweet_vol = tweet_vol.drop_duplicates()
tweet_vol

## Loading, cleaning the sentiment analyzed reddit dataset and meging with the historical bitcoin data 

In [ ]:
cleaned_reddit_posts = pd.read_csv('/content/drive/MyDrive/FYP/dataset/Reddit/reddit_data_sentiment_complete_2021.csv')

In [ ]:
cleaned_reddit_posts

In [ ]:
cleaned_reddit_posts = cleaned_reddit_posts.set_index('timestamp')

In [ ]:
cleaned_reddit_posts

In [ ]:
cleaned_reddit_posts = cleaned_reddit_posts.rename(columns={'reddit_flair': 'flair', 
                                                            'reddit_tb_polarity': 'polarity', 
                                                            'reddit_tb_subjectivity': 'subjectivity',
                                                            'reddit_sid_pos': 'pos',
                                                            'reddit_sid_neg': 'neg',
                                                            'reddit_sid_neu': 'neu',
                                                            'reddit_sid_com': 'compound'})

In [ ]:
cleaned_reddit_posts.index = pd.to_datetime(cleaned_reddit_posts.index)  # convert index to datetime if it's not already
hourly_reddit_posts = cleaned_reddit_posts.resample('H').mean()

In [ ]:
hourly_reddit_posts

In [ ]:
# Remove duplicate rows
hourly_reddit_posts = hourly_reddit_posts.drop_duplicates()

# Remove rows with NaN values
hourly_reddit_posts = hourly_reddit_posts.dropna()

In [ ]:
hourly_reddit_posts

In [ ]:
hourly_reddit_posts.index = hourly_reddit_posts.index.astype(str)


In [ ]:
hourly_reddit_posts

In [ ]:
final_hourly_reddit_bitcoin_merged_df = pd.merge(hourly_reddit_posts, hourly_bitcoin, on='timestamp', how='outer')

In [ ]:
final_hourly_reddit_bitcoin_merged_df

In [ ]:
# Remove duplicate rows
final_hourly_reddit_bitcoin_merged_df = final_hourly_reddit_bitcoin_merged_df.drop_duplicates()

# Remove rows with NaN values
final_hourly_reddit_bitcoin_merged_df = final_hourly_reddit_bitcoin_merged_df.dropna()

In [ ]:
final_hourly_reddit_bitcoin_merged_df

In [ ]:
final_hourly_reddit_bitcoin_merged_df.to_csv('/content/drive/MyDrive/FYP/dataset/Merged Dataset/reddit_bitcoin_merged.csv') 

## Combining the reddit and twitter datasets merged with historical bitcoin data

In [ ]:
import pandas as pd
# Load the dataframes for the two datasets
df_twitter = pd.read_csv('/content/drive/MyDrive/FYP/dataset/Merged Dataset/twitter_bitcoin_merged.csv')
df_reddit = pd.read_csv('/content/drive/MyDrive/FYP/dataset/Merged Dataset/reddit_bitcoin_merged.csv')

df_reddit = df_reddit.drop(columns=['flair', 'compound'])

df_twitter = df_twitter.rename(columns={'Unnamed: 0': 'timestamp'})
df_twitter = df_twitter.drop('compound', axis=1)

In [ ]:
col_names = {'polarity': 'r_polarity', 'subjectivity': 'r_subjectivity', 'pos': 'r_pos', 'neu': 'r_nue', 'neg': 'r_neg', 'open': 'r_open', 'high': 'r_high', 'low': 'r_low', 'close': 'r_close', 'volume': 'r_volume'}
df_reddit = df_reddit.rename(columns=col_names)

In [ ]:
df_reddit

In [ ]:
col_names = {'polarity': 't_polarity', 'subjectivity': 't_subjectivity', 'pos': 't_pos', 'neu': 't_nue', 'neg': 't_neg', 'open': 't_open', 'high': 't_high', 'low': 't_low', 'close': 't_close', 'volume': 't_volume'}
df_twitter = df_twitter.rename(columns=col_names)

In [ ]:
df_twitter


In [ ]:
# Concatenate them horizontally on the timestamp index
df_combined = pd.concat([df_reddit, df_twitter], axis=1)

# Drop the duplicate timestamp column
df_combined = df_combined.loc[:,~df_combined.columns.duplicated()]


In [ ]:
df_combined = df_combined.dropna()


In [ ]:
# combine t_polarity and r_polarity columns
df_combined['polarity'] = (df_combined['t_polarity'] + df_combined['r_polarity']) / 2
# combine t_subjectivity and r_subjectivity columns
df_combined['subjectivity'] = (df_combined['t_subjectivity'] + df_combined['r_subjectivity']) / 2
# combine t_pos and r_pos columns
df_combined['pos'] = (df_combined['t_pos'] + df_combined['r_pos']) / 2
# combine t_neg and r_neg columns
df_combined['neg'] = (df_combined['t_neg'] + df_combined['r_neg']) / 2
# combine t_neu and r_neu columns
df_combined['neu'] = (df_combined['t_nue'] + df_combined['r_nue']) / 2
# combine t_open and r_open columns
df_combined['open'] = (df_combined['t_open'] + df_combined['r_open']) / 2
# combine t_close and r_close columns
df_combined['close'] = (df_combined['t_close'] + df_combined['r_close']) / 2
# combine t_high and r_high columns
df_combined['high'] = (df_combined['t_high'] + df_combined['r_high']) / 2
# combine t_low and r_low columns
df_combined['low'] = (df_combined['t_low'] + df_combined['r_low']) / 2
# combine t_volume and r_volume columns
df_combined['volume'] = (df_combined['t_volume'] + df_combined['r_volume']) / 2

In [ ]:
# drop the original t_polarity and r_polarity columns
df_combined.drop(['t_polarity', 'r_polarity'], axis=1, inplace=True)
# drop the original t_subjectivity and r_subjectivity columns
df_combined.drop(['t_subjectivity', 'r_subjectivity'], axis=1, inplace=True)
# drop the original t_pos and r_pos columns
df_combined.drop(['t_pos', 'r_pos'], axis=1, inplace=True)
# drop the original t_neg and r_neg columns
df_combined.drop(['t_neg', 'r_neg'], axis=1, inplace=True)
# drop the original t_nue and r_nue columns
df_combined.drop(['t_nue', 'r_nue'], axis=1, inplace=True)
# drop the original t_open and r_open columns
df_combined.drop(['t_open', 'r_open'], axis=1, inplace=True)
# drop the original t_close and r_close columns
df_combined.drop(['t_close', 'r_close'], axis=1, inplace=True)
# drop the original t_high and r_high columns
df_combined.drop(['t_high', 'r_high'], axis=1, inplace=True)
# drop the original t_low and r_low columns
df_combined.drop(['t_low', 'r_low'], axis=1, inplace=True)
# drop the original t_volume and r_volume columns
df_combined.drop(['t_volume', 'r_volume'], axis=1, inplace=True)

In [ ]:
# Set the index to be the timestamp column
df_combined = df_combined.set_index('timestamp')

In [ ]:
# drop duplicate rows
df_combined = df_combined.drop_duplicates()

# drop rows with NaN values
df_combined = df_combined.dropna()

In [ ]:
df_combined


In [ ]:
df_combined.to_csv('/content/drive/MyDrive/FYP/dataset/Merged Dataset/reddit_twitter_bitcoin_merged.csv') 
